In [1]:

import lava.lib.dl.slayer as slayer
import matplotlib.pyplot as plt
import numpy as np
import nir
import norse
import tonic
import torch
import tqdm
import importlib
import lava_cnn_reader
importlib.reload(lava_cnn_reader);

In [2]:

bs = 128
collate = tonic.collation.PadTensors(batch_first=False)
to_frame = tonic.transforms.ToFrame(sensor_size=tonic.datasets.NMNIST.sensor_size, time_window=1e3)
test_ds = tonic.datasets.NMNIST("./nmnist", transform=to_frame, train=False)
test_dl = torch.utils.data.DataLoader(test_ds, shuffle=True, batch_size=bs, collate_fn=collate)

In [3]:
graph = nir.read('cnn_sinabs.nir')
norse_graph = norse.torch.from_nir(graph)
children = list(norse_graph.children())
net = lava_cnn_reader.nir_to_lava_dl(graph)

In [11]:
lens = [len(net.blocks), len(children)]
# assert lens[0] == lens[1]

sample_idx = 9020
inp = test_ds[sample_idx][0]
lbl = test_ds[sample_idx][1]

lava_outputs = {}
lava_hidden = {}
norse_outputs = {}
norse_hidden = {}
for idx in range(lens[0]):
    lavablock = net.blocks[idx]
    norseblock = children[[0, 1, 5, 6, 7, 8, 9, 10, 11, 12, 2, 3, 4][idx]]
    print(f'block {idx:02}: {type(lavablock).__name__:10} {type(norseblock).__name__:10}')

    if idx == 0:
        x_lava = torch.Tensor(inp).moveaxis(0, -1).unsqueeze(0)
        x_norse = torch.Tensor(inp)
        print('x_lava', x_lava.shape, 'x_norse', x_norse.shape)

    # lava forward pass
    x = lava_outputs[idx-1] if idx > 0 else x_lava
    if isinstance(lavablock, slayer.neuron.cuba.Neuron):
        out = lavablock.forward(x)
        if isinstance(out, tuple):
            lava_outputs[idx], lava_hidden[idx] = out
        else:
            lava_outputs[idx] = out
        assert torch.allclose(lava_outputs[idx].detach(), lavablock(x).detach())
    else:
        lava_outputs[idx] = lavablock(x)

    print(lava_outputs[idx].shape)
    # print(f'block {idx:02}: {type(lavablock)} {type(norseblock)}')

    #print('close:', torch.allclose(lava_outputs[idx].squeeze(0).moveaxis(-1, 0), norse_outputs[idx], atol=1e-5, rtol=1e-5))

block 00: Conv       Conv2d    
x_lava torch.Size([1, 2, 34, 34, 306]) x_norse torch.Size([306, 2, 34, 34])
torch.Size([1, 16, 16, 16, 306]) torch.Size([306, 16, 16, 16])
block 01: Neuron     IAFCell   
torch.Size([1, 16, 16, 16, 306]) torch.Size([4896, 16, 16])
block 02: Conv       Conv2d    


RuntimeError: Given groups=1, weight of size [16, 16, 3, 3], expected input[1, 4896, 16, 16] to have 16 channels, but got 4896 channels instead